In [1]:
from pathlib import Path
import warnings
import sys
import matplotlib.pyplot as plt

home_path = str(Path.cwd().parent)
sys.path.append(home_path)

sys.path.append('/eos/user/m/morwat/.local/lib/python3.9/site-packages/')

In [2]:
from aper_package.interactive_tool import InteractiveTool

In [7]:
warnings.simplefilter(action='ignore', category=DeprecationWarning)
tool = InteractiveTool(spark=spark)
tool.show()

Loading line from dict:   0%|          | 0/105823 [00:00<?, ?it/s]

Done loading line from dict.           


Loading line from dict:   0%|          | 0/105797 [00:00<?, ?it/s]

Done loading line from dict.           


In [25]:
tool.aperture_data.aper_b1[tool.aperture_data.aper_b1['NAME']=='MQ.21L3.B1']

,NAME,APER_TOL_1,APER_TOL_2,APER_TOL_3,APER_1,APER_2,APER_3,APER_4,MECH_SEP,S
7468,MQ.21L3.B1,0.00084,0.00036,0.0,0.022,0.01715,0.022,0.022,-0.194,5693.193897


In [1]:
import pandas as pd
import numpy as np
import tfs
df_no_nans = tool.aperture_data.aper_b1.dropna()
df_b1 = tfs.read('/eos/project-i02/c/collimation-team/machine_configurations/LHC_run3/2023/MADX/injection/aperture/ap_ir5b1.tfs')
df_b1['NAME']=df_b1['NAME'].str.lower()
df_no_nans['NAME']=df_no_nans['NAME'].str.lower()

NameError: name 'tool' is not defined

In [ ]:
survB1 = pd.read_csv('/eos/project-c/collimation-team/machine_configurations/LHC_run3/2023/MADX/injection/survey_b1.tfs', skiprows=8, header=None, sep='\s+', 
                     names=['NAME','KEYWORD','S','L','ANGLE','X','Y','Z','THETA','PHI','PSI','GLOBALTILT','TILT','SLOT_ID','ASSEMBLY_ID','MECH_SEP','V_POS','COMMENTS'])
survB2 = pd.read_csv('/eos/project-c/collimation-team/machine_configurations/LHC_run3/2023/MADX/injection/survey_b2.tfs', skiprows=8, header=None, sep='\s+', 
                     names=['NAME','KEYWORD','S','L','ANGLE','X','Y','Z','THETA','PHI','PSI','GLOBALTILT','TILT','SLOT_ID','ASSEMBLY_ID','MECH_SEP','V_POS','COMMENTS'])

In [ ]:
merged = pd.merge(df_no_nans, df_b1, on='NAME')
merged['N1x_calculated'] = (merged['APER_1_x']-merged['RTOL']-merged['XTOL']-0.002-merged['X'])/(np.sqrt(1.15*merged['BETX']*tool.aperture_data.epsilon+(merged['DX']**2)*(0.00031**2)))
merged['N1y_calculated'] = (merged['APER_2_x']-merged['RTOL']-merged['YTOL']-0.002-merged['Y'])/(np.sqrt(1.15*merged['BETY']*tool.aperture_data.epsilon+(merged['DY']**2)*(0.00031**2)))

merged2 = pd.merge(df_no_nans, df_b1, on='NAME')
merged2['N1x_calculated'] = (merged['APER_1_x']-merged['RTOL']-merged['XTOL']-0.002-merged['X']+merged['MECH_SEP'])/(np.sqrt(1.15*merged['BETX']*tool.aperture_data.epsilon+(merged['DX']**2)*(0.00031**2)))
merged2['N1y_calculated'] = (merged['APER_2_x']-merged['RTOL']-merged['YTOL']-0.002-merged['Y'])/(np.sqrt(1.15*merged['BETY']*tool.aperture_data.epsilon+(merged['DY']**2)*(0.00031**2)))



In [ ]:
merged[['NAME', 'N1', 'N1x_calculated', 'N1y_calculated']].NAME.values[-1]

In [ ]:
ip5pos = 13329.289216000001

fig,ax = plt.subplots(figsize=(20,15))
ax.plot(merged.S_x-ip5pos,merged.N1)
ax.plot(merged.S_x-ip5pos,np.min(np.array([merged.N1x_calculated.values,merged.N1y_calculated.values]),axis=0))
#ax.plot(merged2.S_x-ip5pos,np.min(np.array([merged2.N1x_calculated.values,merged2.N1y_calculated.values]),axis=0))
ax.set_ylim(0,50)
fig.show()

In [ ]:
merged[merged.NAME.str.contains('mqxa')]

In [ ]:
np.sum(merged['MECH_SEP'])

In [ ]:
merged.iloc[-1]

In [ ]:
def calcaper(ele,df,rtol=1,xtol=1,dp=0.00086,betabeat=1.15,parasitic=True,dpara=0.14,fdqf=2.086,betaarc=170.25):
    row = df[df.NAME==ele]
    dx0 = row['DX']
    dy0 = row['DY']
    if parasitic:
        dx = dx0*(np.sqrt(betabeat)-1) + (betabeat*np.sqrt(row['BETX'])*fdqf*dpara)/np.sqrt(betaarc)
        dy = dy0*(np.sqrt(betabeat)-1) + (betabeat*np.sqrt(row['BETY'])*fdqf*dpara)/np.sqrt(betaarc)
    else:
        dx, dy = dx0, dy0
    aperx = (row['APER_1_x']-rtol*row['RTOL']-xtol*row['XTOL']-0.002-np.abs(row['X']))/(np.sqrt(betabeat*row['BETX']*tool.aperture_data.epsilon+(dx*dp)**2))
    apery = (row['APER_2_x']-rtol*row['RTOL']-xtol*row['YTOL']-0.002-np.abs(row['Y']))/(np.sqrt(betabeat*row['BETY']*tool.aperture_data.epsilon+(dy*dp)**2))
    print([ele,aperx.values[0],apery.values[0],row['N1'].values[0],row['APER_1_x'].values[0],row['RTOL'].values[0],row['XTOL'].values[0],row['BETX'].values[0],row['DX'].values[0]])

In [ ]:
calcaper('mcbh.13r5.b1',merged)
calcaper('mcbh.13r5.b1',merged,rtol=0)
calcaper('mcbh.13r5.b1',merged,xtol=0)
calcaper('mcbh.13r5.b1',merged,dp=3.1e-4)
calcaper('mcbh.13r5.b1',merged,parasitic=True)

In [ ]:
calcaper('mcbh.13r5.b1',merged,betabeat=1.15)
calcaper('mcbh.13r5.b1',merged,betabeat=1.15,)

In [ ]:
tool.aperture_data.epsilon

In [ ]:
3.5e-6*0.938/450

In [ ]:
survB1[survB1.NAME=='mcbh.13r5.b1'.upper()]

In [ ]:
survB1[survB1.NAME=='ip5'.upper()]

In [ ]:
angle2 = -3.212992/2

angle2 = np.arctan2(-8399.576419,-546.205716)

rotMat=np.array([[np.cos(angle2),-np.sin(angle2)],[np.sin(angle2),np.cos(angle2)]])

xzrot = np.dot(rotMat,np.array([-8399.576419,-546.205716]))
print(xzrot)
xzrot[1] - 8409.452148

In [ ]:
angle2 = -3.141593
rotMat=np.array([[np.cos(angle2),-np.sin(angle2)],[np.sin(angle2),np.cos(angle2)]])

np.dot(rotMat,np.array([-8409.452134,0.000027]))

In [ ]:
def rotTrans(data,ref,refMechSep):
    angle=np.arctan2(ref[1],ref[0]+8409.452148/2)
    if((angle<-0.75) & (angle>-0.8)):
        adj=-0.00267
    else:
        adj=0
        
    angle2=-angle+np.pi/2+adj
    
    rotMat=np.array([[np.cos(angle2),-np.sin(angle2)],[np.sin(angle2),np.cos(angle2)]])
    
    temp1=data.copy()
    temp1['X']+=8409.452148/2
    adjust=np.dot(rotMat,np.array([ref[0]+8409.452148/2,ref[1]]))
    temp=np.empty((0,2))
    for index,row in temp1.iterrows():
        row2=np.dot(rotMat,row[['X','Z']])-np.array([adjust[0],adjust[1]-refMechSep])
        temp=np.append(temp,[row2],axis=0)
        
    temp1['X']=temp[:,0]
    temp1['Z']=temp[:,1]
    


    return(temp1)

In [ ]:
ipMechSep=[0,0,-0.224,-0.42,0,0.194,0.224,0]
ipPltRange=[0.22,0.26,0.20,0.34,0.22,0.22,0.22,0.26]
ip = 5
minRange=-400
maxRange=400

b1Orbit=rotTrans(tp.survGetDataRange(survB1,'b1',['IP'+str(ip),minRange],['IP'+str(ip),maxRange],['X','Z','NAME'],'IP'+str(ip),False),
                   tp.survGetDataRange(survB1,'b1',['IP'+str(ip),-25],['IP'+str(ip),+25],
                                    ['X','Z'],'IP'+str(ip),False).loc['IP'+str(ip)].values,
                 ipMechSep[ip-1]/2
                )